<h1 style="color:blue">Purpose:</h1>

When i was younger i used to be a First aid teacher. So i found a real interest in the study of this data set for my road to next rank in kaggle. 

<li>As an introduction i will give some information about Symptoms and what to do because action can save lifes</li>
<li>Look at the data </li>
<li>train 2 models</li>

<p>I want to compare two approachs. One with a blackbox model and an other one with knowledge from my eda. </p>
<p style="color:red">Considere giving an upvote if you enjoy my work </p>

<h1 style="color:blue">Knowledge:</h1>


Heart failure, sometimes known as congestive heart failure, occurs when your heart muscle doesn't pump blood as well as it should. Certain conditions, such as narrowed arteries in your heart (coronary artery disease) or high blood pressure, gradually leave your heart too weak or stiff to fill and pump efficiently.

Not all conditions that lead to heart failure can be reversed, but treatments can improve the signs and symptoms of heart failure and help you live longer. Lifestyle changes — such as exercising, reducing sodium in your diet, managing stress and losing weight — can improve your quality of life.

One way to prevent heart failure is to prevent and control conditions that cause heart failure, such as coronary artery disease, high blood pressure, diabetes or obesity.


<h2 style="color:green">Symptoms:</h2>



Heart failure signs and symptoms may include:

<li>Shortness of breath (dyspnea) when you exert yourself or when you lie down</li>
<li>Fatigue and weakness</li>
<li>Swelling (edema) in your legs, ankles and feet</li>
<li>Rapid or irregular heartbeat</li>
<li>Reduced ability to exercise</li>
<li>Persistent cough or wheezing with white or pink blood-tinged phlegm</li>
<li>Increased need to urinate at night</li>
<li>Swelling of your abdomen (ascites)</li>
<li>Very rapid weight gain from fluid retention</li>
<li>Lack of appetite and nausea</li>
<li>Difficulty concentrating or decreased alertness</li>
<li>Sudden, severe shortness of breath and coughing up pink, foamy mucus</li>
<li>Chest pain if your heart failure is caused by a heart attack</li>
<p></p>


<h2 style="color:green">What to do:</h2>

<li>Not smoking</li>
<li>Controlling certain conditions, such as high blood pressure and diabetes</li>
<li>Staying physically active</li>
<li>Eating healthy foods</li>
<li>Maintaining a healthy weight</li>
<li>Reducing and managing stress</li>


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<h1 style="color:blue">Data analysis:</h1>

In [ ]:
df = pd.read_csv('/kaggle/input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv')
df.head()

In [ ]:
!pip install pywaffle

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from pywaffle import Waffle

In [ ]:
sns.set_style('whitegrid')

In [ ]:
def print_waffle(column):

    equilibre=df[column].value_counts()
    data = equilibre
    fig = plt.figure(
        FigureClass=Waffle, 
        title={'label': column},
        rows=8, 
        values=data, 
        colors=("#232066", "#983D3D"),
        # legend={'loc': 'upper left', 'bbox_to_anchor': (1, 1)},
        icons='child', icon_size=10, 
        icon_legend=True,
        labels=["{0}({1})".format(k,v) for k, v in data.items()],
        legend={'loc':'lower left','bbox_to_anchor':(0,-0.4),'ncol':len(data),'framealpha':0}
    )
    fig.gca().set_facecolor('#EEEEEE')
    fig.set_facecolor('#EEEEEE')
    plt.show()

<h2 style="color:green">Population analysis:</h2>

In [ ]:
print_waffle('DEATH_EVENT')      
print_waffle('diabetes')
print_waffle('smoking')

For this graph i use pywaffle library. Easy to use and nice visual. What i underlign :
<li> Death event is 1/3 of the population</li>
<li> We have the same number of death and smoker (will investigate that later)</li>
<li> Hight proportion of diabetic </li>

In [ ]:
def double_histo_binary(df,column):

    toprint_1=df['DEATH_EVENT'].loc[df[column]==0].value_counts()
    toprint_2=df['DEATH_EVENT'].loc[df[column]==1].value_counts()
    df_tempo=pd.DataFrame({'Not_'+column:toprint_1,column:toprint_2})
    ax=plt.subplot(211)
    df_tempo.plot.bar(title=column+' vs non-'+column +' in absolute value',colormap='Accent',ax=ax)
    ax.set(xlabel='Death_event',ylabel='Count')
    total_Notsmoker=df_tempo['Not_'+column].sum()
    total_smoker=df_tempo[column].sum()
    df_tempo1=pd.DataFrame({'Not_'+column:(df_tempo['Not_'+column].values*100)/total_Notsmoker,column:(df_tempo[column].values*100)/total_smoker})
    ax=plt.subplot(212)
    df_tempo1.plot.bar(title=column+' vs non-'+column +' in %',colormap='Accent',ax=ax)
    ax.set(xlabel='Death_event',ylabel='%')
    
    plt.show()

<h2 style="color:green">Binary variable:</h2>

<p></p>
<p>I want to check if the proportion of death event change for smoker or non smoker for instance.In this steps i will use two graphs: One in absolute value and the other one in %.</p>


In [ ]:
double_histo_binary(df,'smoking')
double_histo_binary(df,'anaemia')
double_histo_binary(df,'diabetes')
double_histo_binary(df,'sex')

Only one really notable variable : anaemia. Seems that if you're in anemia the proportion or death event change. For the other variable i don't see any impact. For my second model i will delete the sex diabete and smoker to see if i have the same accuracy.

<h2 style="color:green">Binary variable:</h2>
<p> here i will compare distribution of variable compared to death event. To achieve my comparaison i will only use histograms.  </p>

In [ ]:
def plot_histogram(column):
    df_death=df.loc[df['DEATH_EVENT']==1]
    df_nodeath=df.loc[df['DEATH_EVENT']==0]
    sns.kdeplot(df_death[column],shade=True,color='red',label='Death_event=1')
    sns.kdeplot(df_nodeath[column],shade=True,color='blue',label='Death_event=0')
    plt.title(column+' distribution by class')
    plt.show()

In [ ]:
plot_histogram('serum_creatinine')
plot_histogram('serum_sodium')
plot_histogram('ejection_fraction')
plot_histogram('platelets')
plot_histogram('high_blood_pressure')
plot_histogram('age')

I will keep four features from here : age,ejection_fraction,serum_sodium and serum_creatrine. In the two others distribution it seems similar and will be difficult use them to separate classes.

In [ ]:
sns.heatmap(df.corr(),linewidths=2,linecolor='black',cmap='viridis')
plt.show()

<h1 style="color:blue">Prepare data:</h1>

In [ ]:
columns_goodplace=['age','anaemia','creatinine_phosphokinase','ejection_fraction','serum_creatinine','serum_sodium','time',
                   'smoking','diabetes','sex','platelets','high_blood_pressure']

In [ ]:
y=df['DEATH_EVENT'].values
y=to_categorical(y)
df.drop(['DEATH_EVENT'],1,inplace=True)
df=df[columns_goodplace]
X=df.values

In [ ]:
scaler=StandardScaler()
ss=scaler.fit(X)
X=scaler.transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=91)

<h1 style="color:blue">Create models:</h1>

in this part i will train two models. One in black box with all the features and an other one with only the variable i decide to keep from data analysis. I will optise the two by a simple brut force of the parameters. I will compare results and will conclude which one is better. 

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
        
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.ylim(top=-0.5,bottom=1+0.5)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

def evaluate_model(history,predicted_classes,y_test,model):
    
#    history.history['acc'][0]=0
#    history.history['val_acc'][0]=0
    print(history)
    fig1, ax_acc = plt.subplots()
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Model - Accuracy')
    plt.legend(['Training', 'Validation'], loc='lower right')
    plt.show()
#    
#    history.history['loss'][0]=0.5
#    history.history['val_loss'][0]=0.5
    fig2, ax_loss = plt.subplots()
    plt.xlabel('Epoch')
    plt.ylabel('loss')
    plt.title('Model- loss')
    plt.legend(['Training', 'Validation'], loc='upper right')
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.show()
    target_names=['0','1']
    
    prediction=predicted_classes
    cnf_matrix = confusion_matrix(y_test, prediction)
    report=classification_report(y_test, prediction, target_names=target_names)
    print(cnf_matrix)
    print(report)
    np.set_printoptions(precision=2)
    plt.figure(figsize=(5, 5))
    plot_confusion_matrix(cnf_matrix, classes=['0', '1'],normalize=True,
                      title='Confusion matrix, with normalization')
    plt.show()
    plt.figure(figsize=(5, 5))
    plot_confusion_matrix(cnf_matrix, classes=['0', '1'],normalize=False,
                      title='Confusion matrix, with normalization')

    plt.show()

In [ ]:
def create_model(Nbr_layer,nbr_per_layer,dropout,batch_size,class_0_weight,class_1_weight,X_train,y_train,X_test,y_test):
    model = Sequential()
    for i in range(Nbr_layer):
        model.add(Dense(nbr_per_layer, activation='relu'))
    model.add(Dropout(dropout))   
    model.add(Dense(2, activation='softmax'))
    model.compile(optimizer="Adam",loss="binary_crossentropy",metrics=["accuracy"])
    
    callbacks = [EarlyStopping(monitor='val_loss', patience=5),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]
    class_weight = {0: class_0_weight,
                    1: class_1_weight}
    
    history=model.fit(X_train,y_train,epochs=50,batch_size=batch_size,validation_data=(X_test,y_test),callbacks=callbacks,class_weight=class_weight)
    model.load_weights('best_model.h5')
    
    
    pred=model.predict(X_test)
    predicted_classes=np.argmax(pred,axis=1)
    y_true=np.argmax(y_test,axis=1)
    # evaluate_model(history,predicted_classes,y_true,model)
    score=f1_score(y_true,predicted_classes)
    return(model,score,history)
    

In [ ]:
layer=[1,2,3]
number_per_layer=[10,20,30,40,60,80]
dropout=[0.1,0.2,0.3,0.4]
batch_size=[8,16,32]
class_0_weight=[1]
class_1_weight=[1]

score=0
best_model=0
best_param=[0,0,0,0]
best_history=0

score_lessdata=0
best_model_lessdata=0
best_param_lessdata=[0,0,0,0]
best_history_lessdata=0

In [ ]:
for element in layer:
    for element1 in number_per_layer:
        for element2 in dropout:
            for element3 in batch_size:
                    for element4 in class_0_weight:
                        for element5 in class_1_weight:
                            print('+++++++++++++++++++++New_train+++++++++++++++++++')
                            print([element,element1,element2,element3,element4,element5])
                            model,score1,history=create_model(element,element1,element2,element3,element4,element5,X_train,y_train,X_test,y_test)
                            if score1>score:
                                score=score1
                                best_model=model
                                best_param=[element,element1,element2,element3,element4,element5]
                                best_history=history
                            model_lessdata,score_lessdata1,history_lessdata=create_model(element,element1,element2,element3,element4,element5,X_train[:,:7],y_train,X_test[:,:7],y_test)
                            if score_lessdata1>score_lessdata:
                                score_lessdata=score_lessdata1
                                best_model_lessdata=model_lessdata
                                best_param_lessdata=[element,element1,element2,element3,element4,element5]
                                best_history_lessdata=history_lessdata
                                                                        


<h2 style="color:green">Black box </h2>

In [ ]:
pred=best_model.predict(X_test)
predicted_classes=np.argmax(pred,axis=1)
y_true=np.argmax(y_test,axis=1)


In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_true,predicted_classes)


In [ ]:
from sklearn.metrics import f1_score
f1_score(y_true,predicted_classes)

In [ ]:
evaluate_model(best_history,predicted_classes,y_true,model)

<h2 style="color:green">With knowledge </h2> 

In [ ]:
pred1=best_model_lessdata.predict(X_test[:,:7])
predicted_classes1=np.argmax(pred1,axis=1)
y_true=np.argmax(y_test,axis=1)
evaluate_model(best_history_lessdata,predicted_classes1,y_true,best_model_lessdata)

In [ ]:
accuracy_score(y_true,predicted_classes1)

In [ ]:
f1_score(y_true,predicted_classes1)

<h1 style="color:blue">Conclusion:</h1>

<p>Model with less data seems to have the same perfomance so we have a gain in computer time.It's seems usefull (sometimes depend of training he is better)</p>

<h1 style="color:blue">Ethik</h1>

<p>thik is a python library for model knoledge. It help to understand as SHAP what is the importance of a variable in a model. I will use this library on my model with more variable to see if i select the correct one in my second model.</p>

In [ ]:
!pip install ethik

In [ ]:
import ethik

In [ ]:
explainer = ethik.ClassificationExplainer()

In [ ]:
X_test1=pd.DataFrame(X_test,columns=columns_goodplace)

In [ ]:
explainer.plot_influence_ranking(
    X_test=X_test1,
    y_pred=predicted_classes,
    n_features=5,
)

We keep this column in our other model. It's an other proof that our choice was relevant.

In [ ]:
explainer.plot_influence(
    X_test=X_test1["ejection_fraction"],
    y_pred=predicted_classes,
)

In [ ]:
explainer.plot_influence(
    X_test=X_test1["age"],
    y_pred=predicted_classes,
)

In [ ]:
bob=X_test1.iloc[1]
Mary=X_test1.iloc[4]

In [ ]:
explainer.plot_influence_comparison(
    X_test=X_test1,
    y_pred=predicted_classes,
    reference=bob,
    compared=Mary,
)


How to analyze this graph ? 
<p>Here, we can see that , on average, people of Mary's age are 10 % more likely to have Heart failure.Luckily people with the serum_creatine of mary 17 % chance to avoid heart failure compare to bob </p>


In [ ]:
y_pred=pd.DataFrame(pred,columns=[0,1])
explainer.plot_influence(
    X_test=X_test1["anaemia"],
    y_pred=y_pred[[0, 1]]
)

In [ ]:
y_pred=pd.DataFrame(pred,columns=[0,1])
explainer.plot_influence(
    X_test=X_test1["high_blood_pressure"],
    y_pred=y_pred[[0, 1]]
)

I will continue this notebook later...